<a href="https://colab.research.google.com/github/Tarig-Mohammed/Data-Science-/blob/main/TaSch_Using_ACO_ALGO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ant Colony Optimization for Task Scheduling

import random
import math

class AntColonyOptimization:
    def __init__(self, tasks, vms, execution_times, num_ants, num_iterations, alpha=1.0, beta=2.0, evaporation_rate=0.5):
        """
        Initialize the Ant Colony Optimization algorithm.

        Args:
            tasks (list): List of task IDs.
            vms (list): List of VM IDs.
            execution_times (dict): Mapping of (task, VM) to execution time.
            num_ants (int): Number of ants in the colony.
            num_iterations (int): Maximum number of iterations.
            alpha (float): Pheromone importance factor.
            beta (float): Heuristic importance factor.
            evaporation_rate (float): Rate at which pheromone evaporates.
        """
        self.tasks = tasks
        self.vms = vms
        self.execution_times = execution_times
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate

        # Initialize pheromone levels
        self.pheromones = {(task, vm): 1.0 for task in tasks for vm in vms}

    def heuristic(self, task, vm):
        """
        Calculate heuristic value (inverse of execution time).
        """
        return 1.0 / self.execution_times[(task, vm)]

    def construct_solution(self):
        """
        Construct a solution for an ant using probabilistic task assignment.
        """
        solution = {}
        for task in self.tasks:
            probabilities = []
            for vm in self.vms:
                pheromone = self.pheromones[(task, vm)] ** self.alpha
                heuristic = self.heuristic(task, vm) ** self.beta
                probabilities.append(pheromone * heuristic)

            # Normalize probabilities
            total = sum(probabilities)
            probabilities = [p / total for p in probabilities]

            # Choose VM based on probabilities
            chosen_vm = random.choices(self.vms, probabilities)[0]
            solution[task] = chosen_vm

        return solution

    def evaluate_solution(self, solution):
        """
        Evaluate a solution by calculating makespan and VM loads.
        """
        vm_loads = {vm: 0 for vm in self.vms}
        for task, vm in solution.items():
            vm_loads[vm] += self.execution_times[(task, vm)]

        makespan = max(vm_loads.values())
        return makespan, vm_loads

    def update_pheromones(self, solutions):
        """
        Update pheromone levels based on the solutions.
        """
        # Evaporate pheromones
        for key in self.pheromones:
            self.pheromones[key] *= (1 - self.evaporation_rate)

        # Reinforce pheromones for good solutions
        for solution, makespan in solutions:
            for task, vm in solution.items():
                self.pheromones[(task, vm)] += 1.0 / makespan

    def run(self):
        """
        Run the ACO algorithm.
        """
        best_solution = None
        best_makespan = float('inf')
        best_vm_loads = None

        for iteration in range(self.num_iterations):
            solutions = []
            for _ in range(self.num_ants):
                solution = self.construct_solution()
                makespan, vm_loads = self.evaluate_solution(solution)
                solutions.append((solution, makespan))

                if makespan < best_makespan:
                    best_solution = solution
                    best_makespan = makespan
                    best_vm_loads = vm_loads
                    print(f"Iteration {iteration + 1}: Best Makespan = {best_makespan}")

            # Update pheromones based on solutions
            self.update_pheromones(solutions)



        return best_solution, best_makespan, best_vm_loads


def parse_input(file_path):
    """
    Parse input file for task scheduling problem.

    Args:
        file_path (str): Path to the input file.

    Returns:
        tuple: (tasks, vms, execution_times)
            tasks: List of task IDs.
            vms: List of VM IDs.
            execution_times: Dictionary mapping (task, VM) to execution time.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse tasks
    tasks = list(map(int, lines[0].strip().split()))

    # Parse VMs
    vms = list(map(int, lines[1].strip().split()))

    # Parse execution times
    execution_times = {}
    for line in lines[2:]:
        entries = line.strip().split()
        for entry in entries:
            task, vm, time = map(int, entry.strip('()').split(','))
            execution_times[(task, vm)] = time

    return tasks, vms, execution_times


def generate_random_input(file_path, n_tasks, n_vms, time_range=(5, 10)):
    """
    Generate a random input file for task scheduling problem in the required format.

    Args:
        file_path (str): Path to save the input file.
        n_tasks (int): Number of tasks.
        n_vms (int): Number of VMs.
        time_range (tuple): Range of execution times (min, max).
    """
    with open(file_path, 'w') as file:
        # Write tasks
        tasks = list(range(1, n_tasks + 1))
        file.write(" ".join(map(str, tasks)) + "\n")

        # Write VMs
        vms = list(range(1, n_vms + 1))
        file.write(" ".join(map(str, vms)) + "\n")

        # Write execution times
        execution_times = []
        for task in tasks:
            for vm in vms:
                time = random.randint(time_range[0], time_range[1])
                execution_times.append(f"({task},{vm},{time})")
        file.write(" ".join(execution_times) + "\n")

# Example usage
if __name__ == "__main__":
    # Generate random input for testing
    random_input_file = "random_input.txt"
    generate_random_input(random_input_file, n_tasks=10, n_vms=5)

    # Parse the input file
    tasks, vms, execution_times = parse_input(random_input_file)
    print("Tasks:", tasks)
    print("VMss:", vms)
    print("Execution Times:", execution_times)

    # Run the ACO algorithm
    aco = AntColonyOptimization(tasks, vms, execution_times, num_ants=10, num_iterations=50)
    best_solution, best_makespan, best_vm_loads = aco.run()

    print("Best Solution:", best_solution)
    print("Best Makespan:", best_makespan)
    print("Makespan of each machine:")
    for vm, load in best_vm_loads.items():
        print(f"VM {vm}: {load}")


Tasks: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
VMss: [1, 2, 3, 4, 5]
Execution Times: {(1, 1): 8, (1, 2): 8, (1, 3): 6, (1, 4): 10, (1, 5): 5, (2, 1): 6, (2, 2): 8, (2, 3): 6, (2, 4): 10, (2, 5): 10, (3, 1): 9, (3, 2): 5, (3, 3): 10, (3, 4): 5, (3, 5): 5, (4, 1): 7, (4, 2): 9, (4, 3): 6, (4, 4): 5, (4, 5): 9, (5, 1): 10, (5, 2): 7, (5, 3): 5, (5, 4): 10, (5, 5): 5, (6, 1): 6, (6, 2): 6, (6, 3): 5, (6, 4): 10, (6, 5): 10, (7, 1): 8, (7, 2): 5, (7, 3): 5, (7, 4): 8, (7, 5): 5, (8, 1): 5, (8, 2): 10, (8, 3): 10, (8, 4): 7, (8, 5): 10, (9, 1): 6, (9, 2): 6, (9, 3): 5, (9, 4): 8, (9, 5): 10, (10, 1): 6, (10, 2): 6, (10, 3): 5, (10, 4): 8, (10, 5): 10}
Iteration 1: Best Makespan = 35
Iteration 1: Best Makespan = 26
Iteration 1: Best Makespan = 16
Iteration 8: Best Makespan = 15
Iteration 11: Best Makespan = 11
Best Solution: {1: 5, 2: 1, 3: 4, 4: 4, 5: 5, 6: 2, 7: 2, 8: 1, 9: 3, 10: 3}
Best Makespan: 11
Makespan of each machine:
VM 1: 11
VM 2: 11
VM 3: 10
VM 4: 10
VM 5: 10


##Discustion

---

### **1. Solution Quality**
The ACO algorithm demonstrates excellent solution quality by iteratively improving the task-to-VM assignment.

- **Dynamic Iterative Refinement**: Starting with a makespan of 35 in the initial solution, the algorithm improves significantly through pheromone updates and heuristic evaluations. By iteration 11, the best makespan is reduced to 11.
- **Convergence**: The solution stabilizes quickly, as the improvements in makespan reduce as the algorithm progresses, reflecting convergence toward a near-optimal or optimal solution.

This result shows that ACO is highly effective for the given task scheduling problem, providing a robust and high-quality solution.

---

### **2. Makespan**
The makespan is the key performance metric for evaluating task scheduling algorithms.

- **Best Makespan**: The best makespan achieved is **11**, which is significantly lower than the initial solutions observed during early iterations (e.g., 35, 26, 16).
- **Load Balancing**: The algorithm reduces the makespan by ensuring a balanced distribution of tasks across VMs. For instance, tasks are distributed so that no VM experiences an excessive load.

The minimization of makespan directly translates into faster completion of all tasks, making this approach highly effective for task scheduling in cloud environments.

---

### **3. Virtual Machine Occupation**
The distribution of tasks among VMs demonstrates good load balancing, which is essential for efficient VM utilization.

- **Balanced Loads**: In the best solution:
  - VM 1 and VM 2 have loads of **11**.
  - VM 3, VM 4, and VM 5 have loads of **10** each.
- This even distribution ensures no VM is over-utilized or under-utilized, leading to efficient resource usage.

- **Task Assignment**: Tasks are assigned to VMs in a manner that minimizes the idle time of VMs. By leveraging both pheromone trails and heuristic information, the algorithm avoids overloading specific VMs, which would lead to inefficiencies.

---

### **Overall Performance**
- **Strengths**:
  - The algorithm provides a high-quality solution with minimal makespan.
  - VM occupation is evenly balanced, ensuring efficient resource usage.
  - The method is iterative and dynamic, making it adaptable to different input sizes and configurations.
  
- **Weaknesses**:
  - While the makespan is minimized, there may be cases where further fine-tuning of ACO parameters (e.g., pheromone evaporation rate or heuristic importance) could yield slightly better results.
  - The algorithm's performance may vary with larger or more complex input sizes.

---

### **Conclusion**
The ACO method effectively minimizes the makespan and achieves balanced VM occupation, making it an excellent choice for task scheduling in cloud computing environments. Its iterative nature and reliance on pheromone and heuristic factors ensure that it produces a high-quality solution while balancing computational efficiency.